In [3]:
import pandas as pd
#!pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
#!pip install sparql-dataframe
import sparql_dataframe
import VizKG.visualize as vkg

In [6]:
endpoint = "http://localhost:3030/Visual_Contagions/sparql"

In [13]:
q = """
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# Number of image, originating from a revue 
# that circulate in other revues. This index quantitatively
# estimates the virality of an image-type computing the 
# number of publications where the same image appears. 
# The higher the number, the higher the virality of an image.

# TODO: 
# 1) image which is earliest in a cluster
# 2) count number of images in cluster
# 3) count number of journal in cluster
# 3) order by number of images and number of journals



SELECT ?cluster ?CL_identifier (MIN(?begin) as ?minBegin) (MAX(?begin) as ?maxBegin) (count(distinct ?image) as ?nOfImages) (count(distinct ?serial) as ?nOfRevues) (count(distinct ?city) as ?nOfCities) (count(distinct ?country) as ?nOfCountries) WHERE {
  
	?cluster a crmdig:D1_Digital_Object;
  		crm:P2_has_type <https://lod.unige.ch/vc/type/cluster> ;
		crm:P148_has_component ?image ;
        crm:P1_is_identified_by ?CL_identifier .

	?image crm:P128i_is_carried_by ?issue .
	?issue crm:P108i_was_produced_by ?production .
	?production frbroo:R23_created_a_realization_of ?serial ;
				crm:P7_took_place_at ?city ;
				crm:P4_has_time-span ?time .

	?time a crm:E52_Time-Span;
		crm:P81a_end_of_the_begin ?begin .

  	?city crm:P89_falls_within ?country .

} 
GROUP BY ?cluster ?CL_identifier
ORDER BY DESC(?nOfCountries) ?minBegin ?maxBegin ?nOfCities ?nOfRevues ?nOfImages
LIMIT 10
"""

In [14]:
df = sparql_dataframe.get(endpoint, q, post=True)

In [15]:
df

,cluster,CL_identifier,minBegin,maxBegin,nOfImages,nOfRevues,nOfCities,nOfCountries
